# Install and import

In [23]:
!pip install -r requirements.txt

In [24]:
# created with love in cooperation with ASPIS
import os
import random
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from scipy.special import expit, logit

from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error

# Settings

In [25]:
# load dataset
dataset = os.listdir("/home/jovyan/data/lightning/ASPIS/datasets/Canada_bins_df_final/")

# SELECT columns, Phi60_Sig1 will be added automatically in function create_windows
# _features_cols = []
# _features_cols = ['f10.7_index','Kp_index', 'Dst_index', 'ap_index']
# _features_cols = ['E','BzGSE','Xbsn','AE','PC','AsyD']
# _features_cols = ['AsyH','SymH','O2[cm^-3]','N2[cm^-3]']
# _features_cols = ['Ti','ne','NmE', 'S4_Sig1','p_Sig1']
# deleted
_features_cols = ['FlowPressure','B','Exospheric_temp[K]', 'nO2+','SI_ind_Sig1','Sun_altitude']

# SELECT bins or use dataset for all bins 
_bins = dataset
# SET window_size
window_size = 45

# SET how many minutes ahead do we want to predict
minutes_ahead = 15

# SET the folder name
folder_name= "full-multinn-5FP-SA"

# please don't change this parameter yet so we can compare on the same dataset
test_year = 2019

path_to_omni = "/home/jovyan/data/lightning/ASPIS/datasets/omniweb/omni4col_INTER_NORMALIZED_hour_2013-2021.parquet.gzip"

f_name = f"{folder_name}/shift-{minutes_ahead}-windows-{window_size}/"
os.makedirs(f_name, exist_ok=True)

# Functions

In [26]:
def create_windows(df, window_size, minutes_ahead, features_cols=_features_cols, rolling_window=None, scintilation_threshold=0.1, path_to_omni_hour=path_to_omni,
                   greater_than_1_to_1=1, look_at_window_scinti=False, y_look_side=3, y_count_win=3):
    """
    Creates windows of the specified size and makes a time-shift.

        Parameters:
                df (Pandas DF): Data frame with features and label
                window_size (int): Features window size (in minutes)
                minutes_ahead (int): By how many minutes to make a prediction
                features_cols (list): List of features to be used for prediction
                rolling_window (int): The size of the label window (minutes) used to aggregate (max) the label
                scintillation_threshold (float): The sigmaPhi threshold from which the signal is considered scintillation
                path_to_omni_hour (string): Path to OMNIWeb hourly date frame
                greater_than_1_to_1 (float): Maximum value of sigmaPhi (values greater than this limit are set to this limit)
                look_at_window_scinti (bool): Decides whether to add feature windows that have scintillation within the feature to the positive class
                y_look_side (int): How many values to the right and left in label to look at to indicate scintillation
                                   (for example, a value of 3 makes a label window of size 7, a value of 2 looks at a label window of size 5)
                y_count_win (int): How many values in the label window must be above the scintillation threshold for a value to be considered a scintillation

        Returns:
                X_scinti (numpy array): Scintillation features windows 
                y_scinti (numpy array): Scintillation label
                X_no_scinti (numpy array): No scintillation features windows
                y_no_scinti (numpy array): No scintillation label
    """

    # return empty arrays, if data frame is too small
    if len(df) < (window_size+minutes_ahead):
        return np.array([]), np.array([]), np.array([]), np.array([])

    # add column 'Phi60_Sig1' if it is not in features or move it to the end
    if "Phi60_Sig1" not in features_cols:
        features_cols.append("Phi60_Sig1")
    else:
        features_cols.remove("Phi60_Sig1")
        features_cols.append("Phi60_Sig1")

    # adds hourly features with hourly resolution
    hourly_features = ["Kp_index", "Dst_index", "ap_index", "f10.7_index"]
    hourly_features_used = []
    minute_features_used = []
    features_minute_resolution = []
    for col in features_cols:
        if col in hourly_features:
            features_minute_resolution.append(False)
            hourly_features_used.append(col)
        else:
            features_minute_resolution.append(True)
            minute_features_used.append(col)

    if False in features_minute_resolution:
        if path_to_omni_hour is not None:
            omni_hour = pd.read_parquet(path_to_omni_hour)
            # fills missing indexes
            new_date_range = pd.date_range(start=df.index[0] - pd.to_timedelta(window_size-1, unit='hour'), end=df.index[-1], freq="min")
            df = df.reindex(new_date_range, fill_value=np.nan)
            omni_hour = omni_hour.loc[str(df.index[0] - pd.to_timedelta(window_size-1, unit='hour')):str(df.index[-1])]
            df[hourly_features_used] = omni_hour[hourly_features_used]
        else:
            raise ValueError('The path to the OMNI hourly dataset is not specified!')
    else:
        # fills missing indexes
        new_date_range = pd.date_range(start=df.index[0], end=df.index[-1], freq="min")
        df = df.reindex(new_date_range, fill_value=np.nan)

    # calculates the number of windows and prepares an empty array
    n_of_windows = len(df) - window_size + 1
    n_of_features = len(features_cols)
    X = np.ones((n_of_windows, n_of_features, window_size))

    # fills the array of features with minute values
    for i in range(-1, (window_size + 1) * -1, -1):
        if i == -1:
            X[:, np.array(features_minute_resolution), i] = X[:, np.array(features_minute_resolution), i] * np.array(df[minute_features_used])[window_size + i:]
        else:
            X[:, np.array(features_minute_resolution), i] = X[:, np.array(features_minute_resolution), i] * np.array(df[minute_features_used])[window_size + i:i + 1]

    # fills the array of features with hour values
    h_m_shift = (window_size-1)*60
    if False in features_minute_resolution:
        features_hour_resolution = [not elem for elem in features_minute_resolution]
        for i in range(0, window_size):
            if i == window_size-1:
                X[h_m_shift:, np.array(features_hour_resolution), i] = X[h_m_shift:, np.array(features_hour_resolution),i] * np.array(df[hourly_features_used])[(i * 60) + i:]
            else:
                X[h_m_shift:, np.array(features_hour_resolution), i] = X[h_m_shift:, np.array(features_hour_resolution), i] * np.array(df[hourly_features_used])[(i*60)+i:((window_size-1-i)*(-60))-(window_size-1-i)]

    # fills the array of labels with values
    if rolling_window is not None:
        df['Phi60_Sig1'] = df['Phi60_Sig1'].rolling(rolling_window, min_periods=1).max()
    if greater_than_1_to_1 is not None:
        df.loc[df['Phi60_Sig1'] > greater_than_1_to_1, 'Phi60_Sig1'] = greater_than_1_to_1
    y = np.array(df['Phi60_Sig1'])[window_size + minutes_ahead - 1:]
    y = np.append(y, np.ones(minutes_ahead) * np.nan)

    # delete windows with NaN values
    X_index_nan = np.argwhere(np.isnan(X))
    y_index_nan = np.argwhere(np.isnan(y))
    rows_with_nan = np.unique(np.append(np.unique(X_index_nan[:, 0]), y_index_nan[:, 0]))
    X = np.delete(X, rows_with_nan, axis=0)
    y = np.delete(y, rows_with_nan, axis=0)

    # splits features and labels into scintillation and non-scintillation windows
    if look_at_window_scinti:
        X_index_scinti = np.argwhere(np.any(X[:, -1] >= scintilation_threshold, axis=1))
    else:
        X_index_scinti = np.argwhere(np.any(X[:, -1] >= np.inf, axis=1))

    if y_look_side is not None and y_count_win is not None:
        Y = np.zeros((len(y), (y_look_side*2)+1))
        Y[:, y_look_side] = y

        for i in range(1, y_look_side + 1):
            Y[:-i, y_look_side + i] = y[i:]

        for i in range(y_look_side):
            Y[y_look_side - i:, i] = y[:-y_look_side + i]

        YY = Y >= scintilation_threshold
        YY = YY.sum(axis=1)
        y_index_scinti = np.argwhere(YY >= y_count_win)
    else:
        y_index_scinti = np.argwhere(y >= scintilation_threshold)

    index_scinti = np.unique(np.append(y_index_scinti, X_index_scinti))
    X_scinti = X[index_scinti]
    y_scinti = y[index_scinti]
    X_no_scinti = np.delete(X, index_scinti, axis=0)
    y_no_scinti = np.delete(y, index_scinti, axis=0)
    
    # column 'Phi60_Sig1' must be dropped because somehow it propagates out of function and normalization doesn't work properly
    features_cols.pop()

    return X_scinti, y_scinti, X_no_scinti, y_no_scinti

In [27]:
def balance_classes(X_train_windows_scinti, y_train_windows_scinti, X_train_windows_no_scinti, y_train_windows_no_scinti):
    
    indexes = random.sample(range((len(X_train_windows_no_scinti))), len(X_train_windows_scinti))
    X_train_windows = np.concatenate((X_train_windows_no_scinti[indexes], X_train_windows_scinti), axis=0)
    y_train_windows = np.concatenate((y_train_windows_no_scinti[indexes], y_train_windows_scinti), axis=0)
    print("total:", len(y_train_windows))

    return X_train_windows, y_train_windows

# Create windows

In [ ]:
X_train_final = []
y_train_final = []
X_test_final = []
y_test_final = []
X_val_final = []
y_val_final = []

# min-max normalization values
max_values = [-np.inf] * len(_features_cols)
min_values = [np.inf] * len(_features_cols)

for bins in _bins:
    print("Calculating normalization values, bin: ", bins)
    df = pd.read_parquet("/home/jovyan/data/lightning/ASPIS/datasets/Canada_bins_df_final/"+bins)
    train = df[df.index.year != test_year]
    
    for i in range(len(_features_cols)):
        
        max_col_value = max(train[_features_cols[i]])
        if max_col_value > max_values[i]:
            max_values[i] = max_col_value
            
        min_col_value = min(train[_features_cols[i]])
        if min_col_value < min_values[i]:
            min_values[i] = min_col_value
print("done")



for bins in _bins:
    # read data
    df = pd.read_parquet("/home/jovyan/data/lightning/ASPIS/datasets/Canada_bins_df_final/"+bins)
    print(bins)
    # normalize
    fn = lambda value, x_max, x_min: (value - x_min) / (x_max - x_min)
    
    for i in range(len(_features_cols)):
        df[_features_cols[i]] = fn(df[_features_cols[i]], max_values[i], min_values[i])
    
    # define train, test and valid subset
    train = df[df.index.year != test_year]
    test = df[df.index.year == test_year]
    
    X_train_windows_scinti, y_train_windows_scinti, X_train_windows_no_scinti, y_train_windows_no_scinti = create_windows(train, window_size, minutes_ahead, scintilation_threshold=0.1)
    
    
    try:
        rng = np.random.default_rng()
        print("lenXtrainBfr")
        print(len(X_train_windows_scinti))
        X_train_windows_scinti = rng.choice(X_train_windows_scinti, size=int(len(X_train_windows_scinti)/8), replace=False)     
        print("lenXtrainAftr")
        print(len(X_train_windows_scinti))
    except:
        print("problemx")

    try:
        rng = np.random.default_rng()
        print("lenytrainBfr")
        print(len(y_train_windows_scinti))
        y_train_windows_scinti = rng.choice(y_train_windows_scinti, size=int(len(y_train_windows_scinti)/8), replace=False)     
        print("lenytrainAftr")
        print(len(y_train_windows_scinti))
    except:
        print("problemy")
    
    
    print("TRAIN set")
    X_train_windows, y_train_windows = balance_classes(X_train_windows_scinti, y_train_windows_scinti, X_train_windows_no_scinti, y_train_windows_no_scinti)
    print("TEST set")
    # y_look_side and y_count_win set to None when creating test windows, because it's a bit faster and we still want all the windows
    X_test_windows, y_test_windows = create_windows(test, window_size, minutes_ahead, scintilation_threshold=0, y_look_side=None, y_count_win=None)[:2]
    
    print("total train")
    X_train_final.extend(X_train_windows)
    print(len(X_train_final))
    y_train_final.extend(y_train_windows)

    print("total test")
    X_test_final.extend(X_test_windows)
    print(len(X_test_final))
    y_test_final.extend(y_test_windows)
    
    print("___________next bin________________")

    
print("done")





name = _features_cols + ["Phi60_Sig1"]

train = {key:[] for key in name}
n=0
print("1")
while n<len(train):
    for col in train:
        for i in X_train_final:
                train[col].append(i[n])
        n = n+1
print("2")
for col in train:
    train[col]=np.vstack(train[col])    
    
test = {key:[] for key in name}
n=0
print("3")
while n<len(test):
    for col in test:
        for i in X_test_final:
                test[col].append(i[n])
        n = n+1
        
for col in test:
    test[col]=np.vstack(test[col])
    
print("4")   
y_test = np.vstack(y_test_final)
y_train = np.vstack(y_train_final)


np.save(f'{f_name}y_test', y_test)
np.save(f'{f_name}y_train', y_train)
np.save(f'{f_name}X_train', train)
np.save(f'{f_name}X_test', test)
print("done")